In [3]:
import os
import torch
import h5py
import pandas as pd
train_h5_path = '/data1/duzhicheng/postive_NSLN/FEATURES_DIRECTORY_2_224_UNI/h5_files'
train_all_image = []
for i in range(len(os.listdir(train_h5_path))):
    train_all_image.append(os.listdir(train_h5_path)[i][:-3]+ ".pt")


# train_all_label = '/data1/duzhicheng/postive_NSLN/label_严格删掉2个淋巴结.xlsx'  # 替换为你的Excel文件的路径
train_all_label = '/data1/duzhicheng/postive_NSLN/label_严格包含2个淋巴结.xlsx'  # 替换为你的Excel文件的路径

df = pd.read_excel(train_all_label)


train_img_list = []
train_label_list= []
train_patient_list = []
for i in range(len(train_all_image)):
    temp = df.loc[df["ID"] == train_all_image[i][:-3], "label"].tolist()
    temp_patient = df.loc[df["ID"] == train_all_image[i][:-3], "patient"].tolist()
    if temp == []:
        continue
    else:
        train_img_list.append(train_all_image[i][:-3])
        train_label_list.append(temp[0])
        train_patient_list.append(temp_patient[0])



#制作特征
import numpy as np
from tqdm import tqdm

train_All_coord_list = []
train_All_feature_list = []
for i in tqdm(range(len(train_img_list))):
    h5file = h5py.File(os.path.join(train_h5_path, train_img_list[i]+".h5"), 'r')
    coords = h5file["coords"]
    faetures = h5file['features']
    train_All_coord_list.append(coords[:])
    train_All_feature_list.append(faetures[:])
    
    
    

#计算患者级别label
train_patients = []
train_patients_label = []
train_patients_features = {}
for i in range(len(train_patient_list)):
    if train_patient_list[i] in train_patients:
        continue
    else:
        train_patients.append(train_patient_list[i])
        train_patients_label.append(train_label_list[i])
        train_patients_features[train_patient_list[i]] = []

for i in range(len(train_patient_list)):
    if train_patient_list[i] in train_patients:
        train_patients_features[train_patient_list[i]].append(train_All_feature_list[i])




###################临床数据
import pandas as pd
train_patients_clinic_data = []
train_patients_wsi_data = []
train_patients_id = []
# 读取CSV文件
file_path = '/data1/duzhicheng/postive_NSLN/clinical.csv'  # 请将此处替换为你的CSV文件路径
df = pd.read_csv(file_path)


for i in range(len(train_patients)):
    # 查找第1列值为123456的行
    target_row = df[df.iloc[:, 0] == train_patients[i]]

    # 如果找到了匹配的行，返回该行的第2到第19列的值，存入列表中
    if not target_row.empty:
        result = target_row.iloc[0, 1:19].tolist()
        train_patients_clinic_data.append(result)
        train_patients_wsi_data.append(train_patients_features[train_patients[i]])
        train_patients_id.append(train_patients[i])
    else:
        print("没有找到行")

train_patients_clinic_data = np.stack(train_patients_clinic_data, axis = 0)

train_clinical_features = train_patients_clinic_data
# wsi_features = patients_features
# cw_features = np.hstack((patients_clinic_data, patients_features))


# all_features = clinical_features
train_labels = np.array(train_patients_label)





100%|██████████| 386/386 [00:20<00:00, 18.71it/s]


In [4]:
import joblib
train_patients_info = {}
train_patients_info["train_patients"] = train_patients
train_patients_info["train_patients_label"] = train_patients_label
joblib.dump(train_patients_info, "train_patients_info.pkl")

['train_patients_info.pkl']

In [5]:
import os
import torch
import h5py
import pandas as pd
test_h5_path = '/data1/duzhicheng/NSLN_shantou/FEATURES_DIRECTORY_2_224_UNI/h5_files'
test_all_image = []
for i in range(len(os.listdir(test_h5_path))):
    test_all_image.append(os.listdir(test_h5_path)[i][:-3]+ ".pt")


test_all_label = '/data1/duzhicheng/NSLN_shantou/label.xlsx'   # 替换为你的Excel文件的路径

df = pd.read_excel(test_all_label)


test_img_list = []
test_label_list= []
test_patient_list = []
for i in range(len(test_all_image)):
    temp = df.loc[df["ID"] == test_all_image[i][:-3], "label"].tolist()
    temp_patient = df.loc[df["ID"] == test_all_image[i][:-3], "patient"].tolist()
    if temp == []:
        continue
    else:
        test_img_list.append(test_all_image[i][:-3])
        test_label_list.append(temp[0])
        test_patient_list.append(temp_patient[0])



#制作特征
import numpy as np
from tqdm import tqdm

test_All_coord_list = []
test_All_feature_list = []
for i in tqdm(range(len(test_img_list))):
    h5file = h5py.File(os.path.join(test_h5_path, test_img_list[i]+".h5"), 'r')
    coords = h5file["coords"]
    faetures = h5file['features']
    test_All_coord_list.append(coords[:])
    test_All_feature_list.append(faetures[:])
    
    
    

#计算患者级别label
test_patients = []
test_patients_label = []
test_patients_features = {}
for i in range(len(test_patient_list)):
    if test_patient_list[i] in test_patients:
        continue
    else:
        test_patients.append(test_patient_list[i])
        test_patients_label.append(test_label_list[i])
        test_patients_features[test_patient_list[i]] = []

for i in range(len(test_patient_list)):
    if test_patient_list[i] in test_patients:
        test_patients_features[test_patient_list[i]].append(test_All_feature_list[i])



###################临床、分类、预后数据
import pandas as pd
test_patients_clinic_data = []
test_patients_wsi_data = []
test_labels = []
test_patients_clinic_name = []
test_patients_dfs_time = []
test_patients_censors = []

# 读取CSV文件
file_path = '/data1/duzhicheng/NSLN_shantou/clinical_shantou_all.xls'  # 请将此处替换为你的CSV文件路径
df = pd.read_excel(file_path)

for i in range(len(test_patients)):
    # 查找第1列值为123456的行
    target_row = df[df.iloc[:, 0] == test_patients[i]]

    # 如果找到了匹配的行，返回该行的第2到第19列的值，存入列表中
    if not target_row.empty:
        result = target_row.iloc[0, 1:19].tolist()
        censor = target_row.iloc[0, 20]
        dfs = target_row.iloc[0, 21]
        test_patients_censors.append(censor) 
        test_patients_dfs_time.append(dfs)
        
        test_patients_clinic_name.append(target_row.iloc[0, 0])
        test_patients_clinic_data.append(result)
        test_patients_wsi_data.append(test_patients_features[test_patients[i]])
        test_labels.append(test_patients_label[i])
    else:
        print(str(test_patients[i]) + " not founnd clinical data!")


test_patients_clinic_data = np.stack(test_patients_clinic_data, axis = 0)

test_clinical_features = test_patients_clinic_data
# wsi_features = patients_features
# cw_features = np.hstack((patients_clinic_data, patients_features))

test_patients_dfs_time = np.stack(test_patients_dfs_time, axis = 0)
test_patients_censors = np.stack(test_patients_censors, axis = 0)

# all_features = clinical_features
test_labels = np.array(test_labels)




100%|██████████| 338/338 [00:16<00:00, 20.52it/s]


139059 not founnd clinical data!
127173 not founnd clinical data!
153841 not founnd clinical data!
159690 not founnd clinical data!
155955 not founnd clinical data!
160810 not founnd clinical data!
166361 not founnd clinical data!
171607 not founnd clinical data!
174130 not founnd clinical data!
183442 not founnd clinical data!
184755 not founnd clinical data!


In [6]:
import joblib
test_patients_info = {}
test_patients_info["test_patients"] = test_patients
test_patients_info["test_patients_label"] = test_patients_label
joblib.dump(test_patients_info, "test_patients_info.pkl")

['test_patients_info.pkl']

病理

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
import random
import os
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import joblib
# 固定seed
import random
import numpy as np
import torch
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # cuDNN's auto-tuner    

from sklearn.model_selection import train_test_split

# 数据已经存在以下变量中
# patients_features: 字典 {患者ID: 对应患者的淋巴结特征列表，每个特征为(n, 1024)}
# patients: 包含所有患者ID的列表
# patients_clinic_data: 临床数据数组 (318×18)
# labels: 患者是否患癌标签 (318×1)
# train_index: 训练集患者序号
# val_index: 验证集患者序号

os.environ['CUDA_VISIBLE_DEVICES']='0'
class AttentionMIL(nn.Module):
    def __init__(self, input_dim):
        super(AttentionMIL, self).__init__()
        self.attention = nn.Linear(input_dim, 1)

    def forward(self, x):
        attention_scores = self.attention(x)  
        attention_weights = torch.softmax(attention_scores, dim=1) 
        weighted_sum = torch.sum(attention_weights * x, dim=1) 
        return weighted_sum

class pathology(nn.Module):
    def __init__(self):
        super(pathology, self).__init__()

        # 降维层
        self.dim_reduction = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 18),
            nn.ReLU()
        )
        # 自注意力层
        self.self_attention = nn.TransformerEncoderLayer(d_model=18, nhead=2, dim_feedforward=18)
        # 分类层
        self.classifier = nn.Sequential(
            nn.Linear(18, 36),
            nn.ReLU(),
            nn.Linear(36, 1),
            nn.Sigmoid()
        )
    
    def forward(self, wsi_features):
        wsi_features = wsi_features.permute(1, 0, 2)
        wsi_features = self.dim_reduction(wsi_features)  
        wsi_features = self.self_attention(wsi_features)
        wsi_features, _ = torch.max(wsi_features,dim =1) 
        output_features = wsi_features
        output = self.classifier(output_features)  # [1, 1]

        return output
    
    
    


In [15]:

for seed in [85] :
    print("*******************************************************")
    print("seed = {}".format(seed))
    
    setup_seed(seed)
    # train_patient_ids = np.arange(len(train_patients_label))
    # # 使用train_test_split进行7:3划分
    # train_index, val_index = train_test_split(train_patient_ids, test_size=0.3, stratify=train_patients_label, random_state=0)
    # # train_index, val_index = train_index, val_index
    train_index = np.array([206, 211, 172, 226, 210, 153,  14,  95, 212, 215, 243, 254, 216,
       182,  71,  23, 168,  98, 107, 108, 169, 148, 131,  72,  81,  10,
       139, 221,  99, 109,   5, 227, 159, 167,  16, 252,  11,  96, 205,
       149, 126, 247,  91, 237,  67, 208, 174, 194, 251,  87, 155, 183,
       158, 171,  76,  49,  61,  21, 141,  33, 121, 189,  60, 187, 137,
        27, 143,   2,  88,  65,  26,   8,  19,  38, 144, 101,  55, 112,
        84,  25, 249, 192,  44, 240, 220, 223,  17, 179,  92,  90, 162,
        35, 235, 129,  20, 142,  40,   6, 242,  93,  42,  75,  53, 253,
        54, 166,  29, 119,   4,  47,  85, 229, 193,  58,  82, 244,  77,
       197,   0, 173, 190,  28, 178,  70, 209,  15, 186, 219, 124, 165,
       207, 230, 185, 170, 122, 191, 225, 145, 214, 233, 203, 136, 138,
        69, 184,  22, 132, 234,  50,  46,  12, 130,  34, 164,  30, 199,
        18, 150,  57, 195, 105,  41, 114, 117, 232, 152, 156,  78, 113,
       133,  79,  74,  83, 202,  31, 140,  89, 147, 146])

    val_index = np.array([224, 204, 116,  62, 188, 198, 154, 103,  48,  43, 201, 111, 200,
       177, 134,  56,  45,  39,  94, 196,  24,  64,  63, 127, 157,  80,
       180, 181, 238,  32, 100, 104,   9,  73, 176, 255, 246, 110, 175,
       160,  97,  86, 120, 128, 217, 228, 250,  51,   7, 245, 163, 115,
       236, 106, 222, 118,  13, 231, 123, 151, 213, 125,  36,  37, 135,
       239,  68, 241, 102,  52, 218,  66, 248,   3, 161,   1,  59])
    # 训练过程
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    AttentionMIL_model = AttentionMIL(input_dim = 1024).to(device)
    pathology_model = pathology().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(list(AttentionMIL_model.parameters()) + list(pathology_model.parameters()), lr=0.001)
    num_epochs = 10
    
    train_loss_his = []
    val_loss_his = []
    loss_his = []
    
    check_point = 0
    for epoch in range(num_epochs):
        # Training phase
        AttentionMIL_model.train()
        pathology_model.train()
        train_label = []
        train_prob = []
        train_split_patients = []
        running_loss = 0.0
        random.shuffle(train_index)
        
        for index in train_index:
            label = torch.tensor(train_labels[index], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
            lymph_node_features_list = train_patients_wsi_data[index]
            wsi_features = []
            for i in range(len(lymph_node_features_list)):
                wsi_features.append(AttentionMIL_model(torch.tensor(lymph_node_features_list[i], dtype=torch.float32).unsqueeze(0).to(device)))
            wsi_features =  torch.stack(wsi_features, dim=0)
            train_split_patients.append(train_patients[index])
            output = pathology_model(wsi_features)
            output.squeeze(0)
            optimizer.zero_grad()
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            loss_his.append(loss.item())
            running_loss += loss.item() 
            train_label.append(label.item())
            train_prob.append(output.item())
            
        train_auc = roc_auc_score(train_label, train_prob)
        train_loss = running_loss / len(train_index)
        train_loss_his.append(train_loss)
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print(f'train_loss: {train_loss:.4f}')
        print(f'train_auc: {train_auc:.4f}')

        #Validation Phase
        with torch.no_grad():
            AttentionMIL_model.eval()
            pathology_model.eval()
            val_label = []
            val_prob = []
            val_patients = []
            running_loss = 0.0
            for index in val_index:
                lymph_node_features_list = train_patients_wsi_data[index]
                label = torch.tensor(train_labels[index], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
                val_patients.append(train_patients[index])
                wsi_features = []
                for i in range(len(lymph_node_features_list)):
                    wsi_features.append(AttentionMIL_model(torch.tensor(lymph_node_features_list[i], dtype=torch.float32).unsqueeze(0).to(device)))
                wsi_features =  torch.stack(wsi_features, dim=0)

                output = pathology_model(wsi_features)
                output.squeeze(0)
                loss = criterion(output, label)
                running_loss += loss.item() 

                val_label.append(label.item())
                val_prob.append(output.item())
                
        val_auc = roc_auc_score(val_label, val_prob)
        val_loss = running_loss / len(val_index)
        val_loss_his.append(val_loss)
        print(f'val_loss: {val_loss:.4f}')
        print(f'val_auc: {val_auc:.4f}')



        if val_auc>check_point:
            check_point = val_auc
            print("保存模型！")


            joblib.dump(train_label, "/data1/duzhicheng/postive_NSLN/model/train_label_pathology_{}.pkl".format(seed))
            joblib.dump(train_prob, "/data1/duzhicheng/postive_NSLN/model/train_prob_pathology_{}.pkl".format(seed))
            joblib.dump(train_loss, "/data1/duzhicheng/postive_NSLN/model/train_loss_pathology_{}.pkl".format(seed))
            joblib.dump(train_split_patients, "/data1/duzhicheng/postive_NSLN/model/train_split_patients_{}.pkl".format(seed))


            joblib.dump(val_label, "/data1/duzhicheng/postive_NSLN/model/val_label_pathology_{}.pkl".format(seed))
            joblib.dump(val_prob, "/data1/duzhicheng/postive_NSLN/model/val_prob_pathology_{}.pkl".format(seed))
            joblib.dump(val_loss, "/data1/duzhicheng/postive_NSLN/model/val_loss_pathology_{}.pkl".format(seed))
            joblib.dump(val_patients, "/data1/duzhicheng/postive_NSLN/model/val_patients_pathology_{}.pkl".format(seed))

            torch.save(AttentionMIL_model.state_dict(), '/data1/duzhicheng/postive_NSLN/model/AttentionMIL_model_clean.pth') 
            torch.save(pathology_model.state_dict(), '/data1/duzhicheng/postive_NSLN/model/pathology_model_clean.pth') 
    
            
            
            
    print("训练过程：")
    train_label  = joblib.load("/data1/duzhicheng/postive_NSLN/model/train_label_pathology_{}.pkl".format(seed))
    train_prob   = joblib.load("/data1/duzhicheng/postive_NSLN/model/train_prob_pathology_{}.pkl".format(seed))
    train_split_patients   = joblib.load("/data1/duzhicheng/postive_NSLN/model/train_split_patients_{}.pkl".format(seed))

    y_test = train_label
    y_prob = train_prob

    # 计算ROC曲线
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    
    
    

    # 计算AUC值
    auc = roc_auc_score(y_test, y_prob)

    # 计算约登指数
    youden_index = tpr - fpr
    best_threshold_index = np.argmax(youden_index)
    best_threshold = thresholds[best_threshold_index]

    # 使用最佳阈值进行分类
    y_pred_best = (y_prob >= best_threshold).astype(int)

    # 计算最佳阈值下的混淆矩阵
    cm = confusion_matrix(y_test, y_pred_best)

    # 计算准确率
    accuracy = accuracy_score(y_test, y_pred_best)

    # 计算敏感度（召回率）
    sensitivity = cm[1, 1] / (cm[1, 1] + cm[1, 0])

    # 计算特异性
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])

    # 打印结果
    print(f"Train AUC: {auc}")
    
    
    
    
    print("验证过程：")
    val_label  = joblib.load("/data1/duzhicheng/postive_NSLN/model/val_label_pathology_{}.pkl".format(seed))
    val_prob   = joblib.load("/data1/duzhicheng/postive_NSLN/model/val_prob_pathology_{}.pkl".format(seed))
    val_patients   = joblib.load("/data1/duzhicheng/postive_NSLN/model/val_patients_pathology_{}.pkl".format(seed))

    y_test = val_label
    y_prob = val_prob

    # 计算ROC曲线
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)

    # 计算AUC值
    auc = roc_auc_score(y_test, y_prob)

    # 计算约登指数
    youden_index = tpr - fpr
    best_threshold_index = np.argmax(youden_index)
    best_threshold = thresholds[best_threshold_index]

    # 使用最佳阈值进行分类
    y_pred_best = (y_prob >= best_threshold).astype(int)

    # 计算最佳阈值下的混淆矩阵
    cm = confusion_matrix(y_test, y_pred_best)

    # 计算准确率
    accuracy = accuracy_score(y_test, y_pred_best)

    # 计算敏感度（召回率）
    sensitivity = cm[1, 1] / (cm[1, 1] + cm[1, 0])

    # 计算特异性
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])

    # 打印结果
    print(f"Val AUC: {auc}")
    print(f"Best Threshold: {best_threshold}")
    print(f"Accuracy: {accuracy}")
    print(f"Sensitivity (Recall): {sensitivity}")
    print(f"Specificity: {specificity}")
    
    
    ########################################################################################################测试阶段
    print("测试过程：")
    
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    AttentionMIL_model = AttentionMIL(input_dim = 1024).to(device)
    pathology_model = pathology().to(device)

    AttentionMIL_model.load_state_dict(torch.load('/data1/duzhicheng/postive_NSLN/model/AttentionMIL_model_clean.pth'))
    pathology_model.load_state_dict(torch.load('/data1/duzhicheng/postive_NSLN/model/pathology_model_clean.pth'))

    criterion = nn.BCELoss()

    test_loss = []
    check_point = 0


    with torch.no_grad():
        AttentionMIL_model.eval()
        pathology_model.eval()
        test_label = []
        test_prob = []
        test_patients_name = []
        running_loss = 0.0
        for index in range(len(test_labels)):
            lymph_node_features_list = test_patients_wsi_data[index]
            
            label = torch.tensor(test_labels[index], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

            wsi_features = []
            for i in range(len(lymph_node_features_list)):
                wsi_features.append(AttentionMIL_model(torch.tensor(lymph_node_features_list[i], dtype=torch.float32).unsqueeze(0).to(device)))
            wsi_features =  torch.stack(wsi_features, dim=0)

            output = pathology_model(wsi_features)
            output.squeeze(0)
            loss = criterion(output, label)
        
            running_loss += loss.item() 

            test_label.append(label.item())
            test_prob.append(output.item())
            test_patients_name.append(test_patients_clinic_name[index])
            
    joblib.dump(test_label, "/data1/duzhicheng/NSLN_shantou/model/test_label_pathology_{}.pkl".format(seed))
    joblib.dump(test_prob, "/data1/duzhicheng/NSLN_shantou/model/test_prob_pathology_{}.pkl".format(seed))
    joblib.dump(test_patients_name, "/data1/duzhicheng/NSLN_shantou/model/test_patients_pathology_{}.pkl".format(seed))
    
    auc = roc_auc_score(test_label, test_prob)
    all_labels,all_outputs = np.array(test_label), np.array(test_prob)
    y_pred_best = (all_outputs >= best_threshold).astype(int)
    # 计算最佳阈值下的混淆矩阵
    cm = confusion_matrix(all_labels, y_pred_best)

    # 计算准确率
    accuracy = accuracy_score(all_labels, y_pred_best)

    # 计算敏感度（召回率）
    sensitivity = cm[1, 1] / (cm[1, 1] + cm[1, 0])

    # 计算特异性
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    print(f"Test AUC: {auc}")
    print(f"Used Threshold: {best_threshold}")
    print(f"Accuracy: {accuracy}")
    print(f"Sensitivity (Recall): {sensitivity}")
    print(f"Specificity: {specificity}")
    
    
    
    
    

*******************************************************
seed = 85
Epoch 0/9
train_loss: 0.6582
train_auc: 0.5198
val_loss: 0.6407
val_auc: 0.8422
保存模型！
Epoch 1/9
train_loss: 0.6487
train_auc: 0.5654
val_loss: 0.6566
val_auc: 0.7696
Epoch 2/9
train_loss: 0.6640
train_auc: 0.4743
val_loss: 0.6405
val_auc: 0.7748
Epoch 3/9
train_loss: 0.6414
train_auc: 0.6099
val_loss: 0.6097
val_auc: 0.7630
Epoch 4/9
train_loss: 0.6347
train_auc: 0.6129
val_loss: 0.6416
val_auc: 0.6696
Epoch 5/9
train_loss: 0.6208
train_auc: 0.6436
val_loss: 0.5738
val_auc: 0.7796
Epoch 6/9
train_loss: 0.5795
train_auc: 0.6832
val_loss: 0.5363
val_auc: 0.8193
Epoch 7/9
train_loss: 0.5983
train_auc: 0.6872
val_loss: 0.6130
val_auc: 0.7274
Epoch 8/9
train_loss: 0.6282
train_auc: 0.5989
val_loss: 0.6444
val_auc: 0.7030
Epoch 9/9
train_loss: 0.6161
train_auc: 0.6372
val_loss: 0.6096
val_auc: 0.7311
训练过程：
Train AUC: 0.5198369565217391
验证过程：
Val AUC: 0.8422222222222222
Best Threshold: 0.37238559126853943
Accuracy: 0.7792207792

In [7]:
import pandas as pd
# 示例列表
    
list1 = train_split_patients
list2 = train_label
list3 = train_prob

# 将列表转换为 DataFrame
df = pd.DataFrame({
    '患者ID': list1,
    "患者标签": list2,
    '预测概率': list3
})

# 保存为 Excel 文件
df.to_excel('训练集（云肿）单病理.xlsx', index=False)

In [7]:
import pandas as pd
# 示例列表
    
list1 = val_patients
list2 = val_label
list3 = val_prob

# 将列表转换为 DataFrame
df = pd.DataFrame({
    '患者ID': list1,
    "患者标签": list2,
    '预测概率': list3
})

# 保存为 Excel 文件
df.to_excel('验证集（云肿）单病理.xlsx', index=False)

In [8]:
import pandas as pd
# 示例列表

    
list1 = [test_patients_name[i] for i in range(len(test_patients_name)) if i not in filter2]
list2 = [test_label[i] for i in range(len(test_label)) if i not in filter2]
list3 = [test_prob[i] for i in range(len(test_prob)) if i not in filter2]

# 将列表转换为 DataFrame
df = pd.DataFrame({
    '患者ID': list1,
    "患者标签": list2,
    '预测概率': list3
})

# 保存为 Excel 文件
df.to_excel('测试集（汕头）单病理.xlsx', index=False)